In [1]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import time
import pydde as d

In [47]:
#Parameters
samplenum = 2
input_size = 3
output_size = 3
time_length = 3; #ms

In [48]:
# Generate simulation
dyn = d.PyDyn('../Data/point-mass_pendulum.sim', time_length)
state_init = dyn.compute(dyn.p_init)
f = dyn.f(state_init, dyn.p_init)
df = dyn.df_dp(state_init, dyn.p_init)
dy = dyn.dy_dp(state_init, dyn.p_init)
#sample p
p = np.ones((3*time_length, samplenum))
p[:,0] = dyn.p_init
if samplenum != 1:
    p[:,1] =p[:,0]
p = torch.tensor(p, requires_grad = True).t()
input = p.double()
print(p)

tensor([[0., 3., 0., 0., 3., 0., 0., 3., 0.],
        [0., 3., 0., 0., 3., 0., 0., 3., 0.]], dtype=torch.float64,
       grad_fn=<TBackward>)


## Building the custon Simulation Activation Function

In [53]:
class Simulate(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, input):
        #print(f'input: {input}')
        p = input.clone().numpy().transpose()
        y_pred = torch.ones([samplenum,3])
        for i in range(len(p[0, :])):
            state = dyn.compute(p[:,i])
            y_pred[i, :] = torch.tensor(state.y[-3:])
        print(f'y_pred: {y_pred}')
        
        ctx.save_for_backward(input)
        
        return y_pred
        
    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        p = input.clone().numpy().transpose()
        dy_dp_batch = torch.zeros([3, 3*time_length])
        for i in range(samplenum):
            state= dyn.compute(p[:, i])
            dy_dp = dyn.dy_dp(state, p[:, i])
            dy_dp = torch.tensor(dy_dp[-3:, :])
            dy_dp_batch = dy_dp_batch + dy_dp
        print(f'shape of dy/dp_batch: {dy_dp_batch.shape}')
        
        grad_input = torch.tensor(grad_output.double().mm(dy_dp_batch))/samplenum
        print(f'shape of grad input: {grad_input.shape}')
        print(f'shape of grad output: {grad_output.shape}')
        print(f'grad_input: {grad_input}')
        print(f'grad_output: {grad_output}')
        return grad_input    

Simulate = Simulate.apply

In [54]:
#GRADCHECK
from torch.autograd import gradcheck

test = gradcheck(Simulate, (input,), eps=1e-3, atol=1e-3, raise_exception = True)
print(test)

y_pred: tensor([[2.1556e-05, 1.9997e+00, 0.0000e+00],
        [2.1556e-05, 1.9997e+00, 0.0000e+00]])
shape of dy/dp_batch: torch.Size([3, 9])
shape of grad input: torch.Size([2, 9])
shape of grad output: torch.Size([2, 3])
grad_input: tensor([[ 8.0987e-03,  1.4527e-06,  0.0000e+00,  5.4188e-03, -3.7828e-06,
          0.0000e+00,  2.7168e-03, -1.9345e-05,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],
       dtype=torch.float64)
grad_output: tensor([[1., 0., 0.],
        [0., 0., 0.]])
shape of dy/dp_batch: torch.Size([3, 9])
shape of grad input: torch.Size([2, 9])
shape of grad output: torch.Size([2, 3])
grad_input: tensor([[ 8.0987e-03,  1.4527e-06,  0.0000e+00,  5.4188e-03, -3.7828e-06,
          0.0000e+00,  2.7168e-03, -1.9345e-05,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],

In [32]:
#print(p.backward(torch.ones(samplenum,9)))
p.requires_grad
2.1556e-05-1.3456e-05

8.100000000000002e-06

In [7]:
from torch.autograd import gradcheck
p = torch.tensor(p, requires_grad = True).t()
input = (p.double())
test = nn.Linear(9,3)
#print(input)
test = gradcheck(test, (input,), eps=1e-3, atol=1e-3, raise_exception = True)
print(test)

RuntimeError: Expected object of scalar type Float but got scalar type Double for argument #2 'mat1' in call to _th_addmm